## Fearless Concurrency: Introduction

<span style="color:skyblue">**Concurrent programming, where different parts of a program execute independently**, and **parallel programming, where different parts of a program execute at the same time**, are becoming increasingly important as more computers take advantage of their multiple processors. Historically, programming in these contexts has been difficult and error prone: Rust hopes to change that.</span>

In Rust, the ownership and type systems are a powerful set of tools to help manage memory safety and concurrency problems. *By leveraging ownership and type checking, many concurrency errors are compile-time errors in Rust rather than runtime errors.*

> Here are the topics we’ll cover in this chapter:
> - How to create threads to run multiple pieces of code at the same time
> - Message-passing concurrency, where channels send messages between threads
> - Shared-state concurrency, where multiple threads have access to some piece of data
> - The `Sync` and `Send` traits, which extend Rust’s concurrency guarantees to user-defined types as well as types provided by the standard library

## Using Threads to Run Code Simultaneously

<span style="color:lightgreen">**In most current operating systems, an executed program’s code is run in a process, and the operating system will manage multiple processes at once. Within a program, you can also have independent parts that run simultaneously in threads**</span>. For example, a web server could have multiple threads so that it could respond to more than one request at the same time. 


> **Concurrency Challenges**
>
> Because threads can run simultaneously, there’s no inherent guarantee about the order in which parts of your code on different threads will run. This can lead to problems, such as:
> - **Race conditions**, where threads are accessing data or resources in an inconsistent order
> - **Deadlocks**, where two threads are waiting for each other, preventing both threads from continuing
> - **Situational bugs** that happen only in certain situations and are hard to reproduce and fix reliably


### Creating a New Thread with `spawn`

**Rust standard library uses a 1:1 model of thread implementation, whereby a program uses one operating system thread per one language thread.**

To create a new thread, we call the `thread::spawn` function and pass it a closure containing the code we want to run in the new thread.


In [8]:
use std::thread;
use std::time::Duration;

fn main() {
    /// Print things from a main and a spawned thread
    /// `thread::sleep` will force a thread to stop its execution for a short duration, 
    /// allowing a different thread to run
    /// The threads will probably take turns, but that isn’t guaranteed: it depends 
    /// on how your operating system schedules the threads. 
    thread::spawn(|| {
        for i in 1..10 {
            println!("hi number {} from the spawned thread!", i);
            thread::sleep(Duration::from_millis(1));
        }
    });

    for i in 1..5 {
        println!("hi number {} from the main thread!", i);
        thread::sleep(Duration::from_millis(2));
    }
}
main()

hi number 1 from the main thread!
hi number 1 from the spawned thread!
hi number 2 from the spawned thread!
hi number 2 from the main thread!
hi number 3 from the spawned thread!
hi number 4 from the spawned thread!
hi number 3 from the main thread!
hi number 5 from the spawned thread!
hi number 6 from the spawned thread!
hi number 4 from the main thread!
hi number 7 from the spawned thread!
hi number 8 from the spawned thread!


()

Note that <span style="color:salmon">***when the main thread of a Rust program completes, all spawned threads are shut down, whether or not they have finished running.***</span>

### Waiting for All Threads to Finish Using `join` Handles

`thread::spawn` returns a `JoinHandle`, which is an owned value that, when we call the `join` method on it, will wait for its thread to finish.

In [41]:
use std::thread;
use std::time::Duration;

fn main() {
    /// Print things from a main and a spawned thread
    /// `thread::sleep` will force a thread to stop its execution for a short duration, 
    /// allowing a different thread to run
    /// The threads will probably take turns, but that isn’t guaranteed: it depends 
    /// on how your operating system schedules the threads. 
    let handle = thread::spawn(|| {
        for i in 1..10 {
            println!("hi number {} from the spawned thread!", i);
            thread::sleep(Duration::from_millis(500));
        }
    });

    for i in 1..5 {
        println!("hi number {} from the main thread!", i);
        thread::sleep(Duration::from_millis(500));
    }
    handle.join().unwrap();  // block the main thread from exiting
}
main()

hi number 1 from the main thread!
hi number 1 from the spawned thread!
hi number 2 from the spawned thread!
hi number 2 from the main thread!
hi number 3 from the spawned thread!
hi number 3 from the main thread!
hi number 4 from the spawned thread!
hi number 4 from the main thread!
hi number 5 from the spawned thread!
hi number 6 from the spawned thread!
hi number 7 from the spawned thread!
hi number 8 from the spawned thread!
hi number 9 from the spawned thread!


()

Let's see what happens when we instead move `handle.join()` before the for loop in main

In [11]:
use std::thread;
use std::time::Duration;

fn main() {
    let handle = thread::spawn(|| {
        for i in 1..10 {
            println!("hi number {} from the spawned thread!", i);
            thread::sleep(Duration::from_millis(1));
        }
    });

    handle.join().unwrap();

    for i in 1..5 {
        println!("hi number {} from the main thread!", i);
        thread::sleep(Duration::from_millis(1));
    }
}

main()

hi number 1 from the spawned thread!
hi number 2 from the spawned thread!
hi number 3 from the spawned thread!
hi number 4 from the spawned thread!
hi number 5 from the spawned thread!
hi number 6 from the spawned thread!
hi number 7 from the spawned thread!
hi number 8 from the spawned thread!
hi number 9 from the spawned thread!
hi number 1 from the main thread!
hi number 2 from the main thread!
hi number 3 from the main thread!
hi number 4 from the main thread!


()

### Using `move` Closures with Threads

<span style="color:lightgreen">*We often use the `move` keyword with closures passed to `thread::spawn` because the closure will then take ownership of the values it uses from the environment, thus transferring ownership of those values from one thread to another.*</span>

In the below function, Rust infers how to capture `v`, and because `println!` only needs a reference to `v`, the closure tries to borrow `v`. However, there’s a problem: <span style="color:orange">*Rust can’t tell how long the spawned thread will run, so it doesn’t know if the reference to `v` will always be valid.*</span>

In [24]:
use std::thread;

fn main() {
    let v = vec![1, 2, 3];

    let handle = thread::spawn(|| {
        println!("Here's a vector: {:?}", v);  // the closure captures v here
    });

    // drop(v); // what if we drop v when the spawned thread is running and having a referece to v?
    
    handle.join().unwrap();
}

main()

Error: closure may outlive the current function, but it borrows `v`, which is owned by the current function

Below, <span style="color:skyblue">*we use the `move` keyword to force a closure to take ownership of the values it uses*</span>

In [30]:
use std::thread;

fn main() {
    let v = vec![1, 2, 3];

    let handle = thread::spawn(move || {
        println!("Here's a vector: {:?}", v);  // move ownership of v to the spawned thread
    });

    // we can't use v anymore in the main thread
    // println!("Here's a vector: {:?}", v);

    handle.join().unwrap();
}

main()

Here's a vector: [1, 2, 3]


()

## Using Message Passing to Transfer Data Between Threads

> **Message Passing**  
> 
> One increasingly popular approach to ensuring safe concurrency is message passing, where threads or actors communicate by sending each other messages containing data. Here’s the idea in a slogan from the Go language documentation: <span style="color:salmon">***“Do not communicate by sharing memory; instead, share memory by communicating.”***</span>

To accomplish message-sending concurrency, Rust's standard library provides an implementation of channels. <span style="color:skyblue">**A channel is a general programming concept by which data is sent from one thread to another.**</span>

A channel has two halves: <span style="color:skyblue">**a transmitter and a receiver.**</span>
- One part of your code calls methods on the transmitter with the data you want to send, and another part checks the receiving end for arriving messages
- A channel is closed if either the transmitter or receiver half is dropped

<span style="color:lightgreen">**Example program**</span>: a program with one thread to generate values and send them down a channel, and another thread that will receive the values and print them out

In [38]:
use std::sync::mpsc;
use std::time::Duration;

fn main() {
    // mpsc: multiple producer, single consumer (multiple streams flow into one river)
    // tx: the transmitter
    // rx: the receiver
    let (tx, rx) = mpsc::channel();

    // move tx into a new thread and use .send() to send a "hi" string
    thread::spawn(move || {
        let val = String::from("hi");
        thread::sleep(Duration::from_secs(2)); // do something for 2s
        tx.send(val).unwrap();  // send "hi"
    });

    // Receiving the value “hi” in the main thread and printing it
    let received = rx.recv().unwrap();  // recv() blocks the main thread’s execution 
                                        // and wait until a value is sent down the channel
    println!("Got: {}", received);
}

main()

Got: hi


()

📓 Note: Instead of `recv()`, we can use `try_recv()` method that doesn’t block, but will instead return a `Result<T, E>` immediately: an `Ok` value holding a message if one is available and an `Err` value if there aren’t any messages this time. Using `try_recv` is useful if this thread has other work to do while waiting for messages: we could write a loop that calls `try_recv` every so often, handles a message if one is available, and otherwise does other work for a little while until checking again.

### Channels and Ownership Transference

<span style="color:lightgreen">*The `send` function takes ownership of its parameter, and when the value is moved, the receiver takes ownership of it*</span>

In [37]:
use std::sync::mpsc;
use std::thread;

fn main() {
    let (tx, rx) = mpsc::channel();

    thread::spawn(move || {
        let val = String::from("hi");
        tx.send(val).unwrap();
        println!("val is {}", val);  // try to use val here and it won't work
    });

    let received = rx.recv().unwrap();
    println!("Got: {}", received);
}

Error: borrow of moved value: `val`

### Sending Multiple Values and Seeing the Receiver Waiting
Now, the spawned thread will now send multiple messages and pause for a second between each message

In [45]:
use std::sync::mpsc;
use std::thread;
use std::time::Duration;

fn main() {
    let (tx, rx) = mpsc::channel();

    thread::spawn(move || {
        let vals = vec![
            String::from("hi"),
            String::from("from"),
            String::from("the"),
            String::from("thread"),
        ];

        for val in vals {
            tx.send(val).unwrap();
            thread::sleep(Duration::from_secs(1));
        }
    });

    for received in rx {
        println!("Got: {}", received);
    }
}

main()

Got: hi
Got: from
Got: the
Got: thread


()

### Creating Multiple Producers by Cloning the Transmitter

In [48]:
use std::sync::mpsc;
use std::thread;
use std::time::Duration;

fn main() {
    // --snip--

    let (tx, rx) = mpsc::channel();

    let tx1 = tx.clone();  // a new transmitter
    thread::spawn(move || {
        let vals = vec![
            String::from("hi"),
            String::from("from"),
            String::from("the"),
            String::from("thread"),
        ];

        for val in vals {
            tx1.send(val).unwrap();
            thread::sleep(Duration::from_secs(1));
        }
    });

    thread::spawn(move || {
        let vals = vec![
            String::from("more"),
            String::from("messages"),
            String::from("for"),
            String::from("you"),
        ];

        for val in vals {
            tx.send(val).unwrap();
            thread::sleep(Duration::from_secs(1));
        }
    });

    for received in rx {
        println!("Got: {}", received);
    }

    // --snip--
}

main()

Got: hi
Got: more
Got: from
Got: messages
Got: the
Got: for
Got: thread
Got: you


()

## Shared-State Concurrency

<span style="color:orange">*Message passing is a fine way of handling concurrency, but it’s not the only one. Another method would be for **multiple threads to access the same shared data** (do this with caution).*</span>

> <span style="color:skyblue">**Concurrency Challenges**</span>
>
> - <span style="color:skyblue">*Channels - Single Onwership*</span>: In a way, channels in any programming language are similar to single ownership, because once you transfer a value down a channel, you should no longer use that value
> - <span style="color:skyblue">*Shared Memory - Multiple Ownership*</span>: multiple threads can access the same memory location at the same time. For example, multiple ownership with smart pointers

### Using Mutexes to Allow Access to Data from One Thread at a Time

<span style="color:skyblue">*Mutex is an abbreviation for mutual exclusion, as in, a mutex allows only one thread to access some data at any given time*</span>. To access the data in a mutex, a thread must first signal that it wants access by asking to acquire the mutex’s lock. The lock is a data structure that is part of the mutex that keeps track of who currently has exclusive access to the data. Therefore, the mutex is described as guarding the data it holds via the locking system. 

To use mutexes, you have to remember two rules:
1. You must attempt to acquire the lock before using the data.
2. When you’re done with the data that the mutex guards, you must unlock the data so other threads can acquire the lock.



#### The API of `Mutex<T>`

In [50]:
use std::sync::Mutex;

fn main() {
    let m: Mutex<i32> = Mutex::new(5);  // create a Mutex of an i32 value 

    {
        let mut num: i32 = m.lock().unwrap();  // acquire the lock to access the data inside the mutex
                                               // lock() would fail if another thread holding the lock panicked
        *num += 1;
    }

    println!("m = {:?}", m);
}

main()

m = Mutex { data: 6, poisoned: false, .. }


()

- `Mutex<T>` is a smart pointer.  
- `Mutex<T>.lock()` returns a smart pointer called `MutexGuard`.  
- `MutexGuard` is wrapped in a `LockResult` that we handled with `unwrap()`.   
- The `MutexGuard` smart pointer implements `Deref` to point at our inner data.  
- `MutexGuard` also has a `Drop` implementation that releases the lock automatically when a `MutexGuard` goes out of scope, so we don’t risk forgetting to release the lock and blocking the mutex from being used by other threads, because the lock release happens automatically.

#### Sharing a `Mutex<T>` Between Multiple Threads

Now, let’s try to share a value between multiple threads using `Mutex<T>`. We’ll spin up 10 threads and have them each increment a counter value by 1, so the counter goes from 0 to 10

In [51]:
use std::sync::Mutex;
use std::thread;

fn main() {
    let counter: Mutex<i32> = Mutex::new(0);
    let mut handles = vec![];

    for _ in 0..10 {
        // moves the `counter`` into the thread, 
        // acquires a lock on the Mutex<T> by calling the lock method, 
        // and then adds 1 to the value in the mutex
        let handle = thread::spawn(move || {
            let mut num = counter.lock().unwrap();
            *num += 1;
        });
        handles.push(handle);
    }

    // wait for the threads to finish in the main thread
    for handle in handles {
        handle.join().unwrap();
    }

    println!("Result: {}", *counter.lock().unwrap());
}

Error: borrow of moved value: `counter`

We got the error message saying that the `counter` value was moved in the previous iteration of the loop. Rust is telling us that <span style="color:orange">*we can’t move the ownership of lock `counter` into multiple threads*</span>

#### Multiple Ownership with Multiple Threads

In Chapter 15, we gave a value multiple owners by using the smart pointer `Rc<T>` to create a reference counted value. Let's try wraping the `Mutex<T>` in `Rc<T>` and clone the `Rc<T>` before moving ownership to a new thread

In [52]:
use std::rc::Rc;
use std::sync::Mutex;
use std::thread;

fn main() {
    let counter = Rc::new(Mutex::new(0));
    let mut handles = vec![];

    for _ in 0..10 {
        let counter = Rc::clone(&counter);
        let handle = thread::spawn(move || {
            let mut num = counter.lock().unwrap();

            *num += 1;
        });
        handles.push(handle);
    }

    for handle in handles {
        handle.join().unwrap();
    }

    println!("Result: {}", *counter.lock().unwrap());
}

Error: `Rc<Mutex<i32>>` cannot be sent between threads safely

We got the error `Rc<Mutex<i32>> cannot be sent between threads safely.`

Unfortunately, `Rc<T>` is not safe to share across threads. When `Rc<T>` manages the reference count, it adds to the count for each call to `clone` and subtracts from the count when each clone is dropped. However, <span style="color:orange">*`Rc<T>` doesn’t use any concurrency primitives to make sure that changes to the count can’t be interrupted by another thread. This could lead to wrong counts—subtle bugs that could in turn lead to memory leaks or a value being dropped before we’re done with it.*</span>   
<span style="color:lightgreen">*What we need is a type exactly like `Rc<T>` but one that makes changes to the reference count in a thread-safe way.*</span>



#### Atomic Reference Counting with `Arc<T>`

`Arc<T>` is a type like `Rc<T>` that is safe to use in concurrent situations. The a stands for atomic, meaning it’s an atomically reference counted type.

You might wonder why standard library types aren’t implemented to use `Arc<T>` by default. The reason is that thread safety comes with a performance penalty that you only want to pay when you really need to.

`Arc<T>` and `Rc<T>` have the same API:

In [53]:
use std::sync::{Arc, Mutex};
use std::thread;

fn main() {
    let counter = Arc::new(Mutex::new(0));
    let mut handles = vec![];

    for _ in 0..10 {
        let counter = Arc::clone(&counter);
        let handle = thread::spawn(move || {
            let mut num = counter.lock().unwrap();
            *num += 1;
        });
        handles.push(handle);
    }

    for handle in handles {
        handle.join().unwrap();
    }

    println!("Result: {}", *counter.lock().unwrap());
}

main()

Result: 10


()

> <span style="color:green">**Thread-safe 1-10 Counting, and More**</span>
>
> <span style="color:lightgreen">*We did it! With the above program, we counted from 0 to 10, which may not seem very impressive, but it did teach us a lot about `Mutex<T>` and thread safety. You could also use this program’s structure to do more complicated operations than just incrementing a counter. Using this strategy, you can **divide a calculation into independent parts, split those parts across threads, and then use a `Mutex<T>` to have each thread update the final result with its part.***</span>

### Similarities Between `RefCell<T>`/`Rc<T>` and `Mutex<T>`/`Arc<T>`


In the same way we used `RefCell<T>` in Chapter 15 to allow us to mutate contents inside an `Rc<T>`, *we use `Mutex<T>` to mutate contents inside an `Arc<T>`.*

> <span style="color:orange">**Deadlocks with `Mutex<T>`**</span>
>
> <span style="color:orange">*`Mutex<T>` comes with the risk of creating deadlocks. These occur when an operation needs to lock two resources and two threads have each acquired one of the locks, causing them to wait for each other forever*</span>

## Extensible Concurrency with the `Sync` and `Send` Traits



### Allowing Transference of Ownership Between Threads with `Send`

The `Send` marker trait indicates that ownership of values of the type implementing `Send` can be transferred between threads. 

Almost every Rust type is `Send`, but there are some exceptions, including `Rc<T>`: this cannot be `Send` because if you cloned an `Rc<T>` value and tried to transfer ownership of the clone to another thread, both threads might update the reference count at the same time. For this reason, `Rc<T>` is implemented for use in single-threaded situations where you don’t want to pay the thread-safe performance penalty.

### Allowing Access from Multiple Threads with `Sync`
The `Sync` marker trait indicates that it is safe for the type implementing `Sync` to be referenced from multiple threads. In other words, any type `T` is `Sync` if `&T` (an immutable reference to `T`) is `Send`, meaning the reference can be sent safely to another thread. Similar to `Send`, primitive types are `Sync`, and types composed entirely of types that are `Sync` are also `Sync`.

`Rc<T>, RefCell<T>` and the family of related `Cell<T>` types are not `Sync`.

### Implementing `Send` and `Sync` Manually Is Unsafe

Because <span style="color:lightgreen">*types that are made up of `Send` and `Sync` traits are automatically also `Send` and `Sync`*</span>, we don’t have to implement those traits manuall. As marker traits, they don’t even have any methods to implement. They’re just useful for enforcing invariants related to concurrency.

<span style="color:orange">*Manually implementing these traits involves implementing unsafe Rust code.*</span> Building new concurrent types not made up of `Send` and `Sync` parts requires careful thought to uphold the safety guarantees.

## Summary

Since very little of how Rust handles concurrency is part of the language, <span style="color:lightgreen">*many concurrency solutions are implemented as crates*</span>.

The Rust standard library provides *channels for message passing* and *smart pointer types, such as `Mutex<T>` and `Arc<T>`, that are safe to use in concurrent contexts.*

<span style="color:lightgreen">*The type system and the borrow checker ensure that the code using these solutions won’t end up with data races or invalid references. **Once you get your code to compile, you can rest assured that it will happily run on multiple threads without the kinds of hard-to-track-down bugs common in other languages**.*</span>